In [124]:
import requests
import json
import os
import time

from functools import wraps
from pathlib import Path
from enum import Enum

In [2]:
bearer = 'Bearer AAAAAAAAAAAAAAAAAAAAAJQ4TQEAAAAAi1uW5CqJ70nIsAQBjocmatM8MkY%3DCuxVktwfJ6nz7ceUh2w3QuIshTlJjWzMOxxbg4nlgZXJnciXyW'

In [149]:
url = "https://api.twitter.com/2/tweets/search/stream"
ROOT_FOLDER = r"C:\Users\Orion\Documents\OutputTweets"
MAX_SIZE = 500000
params = {
    "tweet.fields": "entities,public_metrics",
    "expansions": "author_id,in_reply_to_user_id,attachments.media_keys",
    "media.fields": "url",
    "user.fields": "id,verified"
}

In [151]:
s = requests.Session()
s.headers.update({"Authorization": bearer})

In [161]:
class Media(Enum):
    PHOTO = 1
    VIDEO = 2
    GIF = 3
    
    def __eq__(self, other):
        if isinstance(other, str):
            return self.name.lower() == other

In [152]:
def timeit(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        t1 = time.time()
        res = f(*args, **kwargs)
        print(f"{f.__name__} executed in {time.time() - t1}")
        return res
    return wrapper

In [167]:
# test_handle_tweet
tweet = """
{
    "data": {
        "attachments": {
            "media_keys": [
                "3_1437787186561228800"
            ]
        },
        "author_id": "21313364",
        "entities": {
            "urls": [
                {
                    "start": 129,
                    "end": 152,
                    "url": "https://t.co/1kUyRMrCmS",
                    "expanded_url": "https://www.nouvelobs.com/droits-des-femmes/20210914.OBS48633/il-y-a-un-afflux-considerable-de-demandes-le-gouvernement-veut-reduire-les-delais-d-attente-pour-la-pma-d-un-an-a-six-mois.html?utm_term=Autofeed&utm_medium=Social&utm_source=Twitter#Echobox=1631630086",
                    "display_url": "nouvelobs.com/droits-des-fem…",
                    "images": [
                        {
                            "url": "https://pbs.twimg.com/news_img/1437787191103692800/Wg5T_O5D?format=jpg&name=orig",
                            "width": 1024,
                            "height": 512
                        },
                        {
                            "url": "https://pbs.twimg.com/news_img/1437787191103692800/Wg5T_O5D?format=jpg&name=150x150",
                            "width": 150,
                            "height": 150
                        }
                    ],
                    "status": 200,
                    "title": "« Il y a un afflux considérable de demandes » : le gouvernement veut réduire les délais d’attente pour la PMA d’un an à six mois",
                    "description": "Le Parlement a voté fin juin la procréation médicalement assistée pour toutes les femmes, qu’elles soient hétérosexuelles, homosexuelles ou monoparentales. Une enveloppe de 8 millions d’euros doit permettre de répondre à l’afflux de demandes.",
                    "unwound_url": "https://www.nouvelobs.com/droits-des-femmes/20210914.OBS48633/il-y-a-un-afflux-considerable-de-demandes-le-gouvernement-veut-reduire-les-delais-d-attente-pour-la-pma-d-un-an-a-six-mois.html?utm_term=Autofeed&utm_medium=Social&utm_source=Twitter#Echobox=1631630086"
                },
                {
                    "start": 153,
                    "end": 176,
                    "url": "https://t.co/1uGLGEP38c",
                    "expanded_url": "https://twitter.com/lobs/status/1437787188868091906/photo/1",
                    "display_url": "pic.twitter.com/1uGLGEP38c"
                }
            ]
        },
        "id": "1437787188868091906",
        "organic_metrics": {},
        "text": "« Il y a un afflux considérable de demandes » : le gouvernement veut réduire les délais d’attente pour la PMA d’un an à six mois https://t.co/1kUyRMrCmS https://t.co/1uGLGEP38c"
    },
    "includes": {
        "media": [
            {
                "media_key": "3_1437787186561228800",
                "type": "photo",
                "url": "https://pbs.twimg.com/media/E_QL1BWXIAAEEPV.jpg"
            }
        ],
        "users": [
            {
                "id": "21313364",
                "name": "L'Obs",
                "username": "lobs"
            }
        ]
    },
    "matching_rules": [
        {
            "id": 1437707103754506241,
            "tag": "PMA"
        }
    ]
}"""
handle_tweet(json.loads(tweet))


handle_tweet executed in 0.2706904411315918


In [166]:
def handle_media(tweet):
    # check if there are some media in the tweet
    if not tweet.get('includes', {}).get('media', False):
        return
    # extract the media
    media = tweet['includes']['media']
    for medium in media:
        if medium['type'] == Media.PHOTO:
            handle_photo(**medium)
        else:
            print(f"unhandled media type currently: {medium['type']}")

In [168]:
def handle_photo(media_key=None, url=None, **kwargs):
    """
    save a picture to disk
    pictures are saves in root_directory/images and unlike tweet are not saved by tags
    # TODO: add metadata + sha1 + occurences count
    # TODO: split the download part to use it for every media
    """
    if not media_key or not url:
        raise ValueError("Missing field when trying to save image")
    file_type = url.split('.')[-1]
    file_name = f"{media_key}.{file_type}"
    try:
        res = requests.get(url)
        print("downloading image")
    except requests.RequestException:
        raise ValueError(f"There was an error when downloading the image with following url: {url}, please check your connection or url")
    # save to disk
    # TODO: maybe ensure the creation of 'images' elsewhere ? 
    directory = Path(os.path.join(ROOT_FOLDER, 'images'))
    directory.mkdir(parents=True, exist_ok=True)
    with open(os.path.join(directory, file_name), 'wb') as f:
        f.write(res.content)

In [155]:
@timeit
def handle_tweet(tweet):
    id = tweet['data']['id']
    
    # filter tags:
    tags = list(set([r['tag'] for r in tweet["matching_rules"]]))
    
    # save media:
    handle_media(tweet)
    
    # save collected tweet in every tag
    for tag in tags:
        # create directory if not exist
        directory = Path(os.path.join(ROOT_FOLDER), tag)
        directory.mkdir(parents=True, exist_ok=True)
        with open(os.path.join(directory, f"{id}.json"), 'w', encoding='utf-8') as f:
            json.dump(tweet, f, indent=4, ensure_ascii=False)

In [24]:
def get_folder_size(path):
    return sum(f.stat().st_size for f in Path(path).glob('**/*') if f.is_file())

In [110]:
@timeit
def has_free_space():
    if get_folder_size(ROOT_FOLDER) < MAX_SIZE:
        return True
    raise OSError("The maxsize of the storage directory has been reached")

In [169]:
with s.get(url, params=params, stream=True, timeout=5000) as resp:
    if resp.status_code != 200:
        print(f"error {resp.status_code}")
        print(resp.content)
    for line in resp.iter_lines():
        if line and has_free_space():
            print(line.decode("utf-8"))
            handle_tweet(json.loads(line.decode("utf-8")))
        else:
            print("waiting for new tweets")

has_free_space executed in 0.04297780990600586
{"data":{"attachments":{},"author_id":"1313158124149395456","entities":{"mentions":[{"start":3,"end":12,"username":"jvivante","id":"561109226"}]},"id":"1437802785806893064","public_metrics":{"retweet_count":30,"reply_count":0,"like_count":0,"quote_count":0},"text":"RT @jvivante: Entre deux inepties historiques, comme affirmer que Pétain a protégé les Juifs de France, Zemmour ne parle de rien d'autre qu…"},"includes":{"users":[{"id":"1313158124149395456","name":"Sid - Vacciné - #Anti-Extrêmes - #Macron2022","username":"Sid96444775","verified":false},{"id":"561109226","name":"Jimmy Vivante","username":"jvivante","verified":false}]},"matching_rules":[{"id":1437409111315042307,"tag":"GR"}]}
handle_tweet executed in 0.0010006427764892578
waiting for new tweets
waiting for new tweets
has_free_space executed in 0.06453299522399902
{"data":{"attachments":{},"author_id":"824353266406477834","entities":{"mentions":[{"start":3,"end":12,"username":"jv

waiting for new tweets
has_free_space executed in 0.06199908256530762
{"data":{"attachments":{},"author_id":"1008025140","entities":{"mentions":[{"start":3,"end":16,"username":"norbertnabet","id":"430643563"}],"urls":[{"start":117,"end":140,"url":"https://t.co/q3G1JcYoa0","expanded_url":"https://tetu.com/2021/09/14/info-tetu-olivier-veran-promet-8-millions-deuros-pour-aider-louverture-de-la-pma/","display_url":"tetu.com/2021/09/14/inf…","images":[{"url":"https://pbs.twimg.com/news_img/1437740741829218307/veLZb-uo?format=png&name=orig","width":1934,"height":1062},{"url":"https://pbs.twimg.com/news_img/1437740741829218307/veLZb-uo?format=png&name=150x150","width":150,"height":150}],"status":200,"title":"Ouverture de la PMA : Olivier Véran promet 8 millions d'euros pour répondre à l'afflux de demandes - TÊTU","description":"INFO TÊTU - Le ministère de la Santé promet 8 millions d'euros pour les nouveaux projets de PMA. Olivier Véran promet un délai d'attente abaissé à six mois.","unwound_

OSError: The maxsize of the storage directory has been reached